In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

Using Theano backend.


In [1]:
import trading as trd
from trading import StockHolding, State

In [9]:
# relaod module after modifying it
import importlib
importlib.reload(trd.stock_history)
importlib.reload(trd.portfolio)
importlib.reload(trd.benchmarks)
importlib.reload(trd.environment)
importlib.reload(trd)

<module 'trading' from 'C:\\Users\\hamza\\Documents\\school\\cs_229\\cs229_final_project\\src\\trading\\__init__.py'>

## Load Data

In [10]:
train, test = trd.get_stock_pairs(20)

## Formulate Problem

In [8]:
# number of outputs
k = actions.size

trans_cost = 0.01

In [14]:
model = Sequential([
    Dense(output_dim=64, input_dim=n, init='uniform'),
    Activation('sigmoid'),
    Dense(output_dim=k)]) #Activation("softmax")

NameError: name 'n' is not defined

In [8]:
# momentum in [0.5, 0.9, 0.95, 0.99]
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

In [9]:
model.fit(data, y, nb_epoch=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 31.0667 - acc: 0.3550     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 22.0609 - acc: 0.3470     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 22.2079 - acc: 0.3350     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 21.9613 - acc: 0.3360     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 21.9638 - acc: 0.3250     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 21.8397 - acc: 0.3230     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 21.7023 - acc: 0.3810     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 21.6407 - acc: 0.3670     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 21.6806 - acc: 0.3440     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 21.6966 - acc: 0.3340     


In [12]:
model.predict(np.random.rand(2, n))

array([[ 7.62245607,  7.38628244,  7.45234728],
       [ 8.50828552,  8.19099331,  8.26092625]], dtype=float32)